# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [2]:
import sqlite3    # Python interface for SQLite databases

In [3]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [4]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)


From this point we will use __%%sql__ magic

In [5]:
%load_ext sql
%sql sqlite:///wine.db

u'Connected: @wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [6]:
%%sql SELECT *
FROM Master1

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [7]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

In [9]:
%%sql 
select DEGRE, CRU, MIN(NV) as min1, MAX(NV) as max1 from master1 group by DEGRE, CRU HAVING min1 <> max1 

 * sqlite:///wine.db
Done.


DEGRE,CRU,min1,max1
None,Savigny les Beaunes,22,39
11,Chiroubles,46,47
11,Palette,71,90
11,Saint Chinian,96,97
11,Seyssel,79,100
11,Tavel,66,86


From table Master1, find maximum fonctionnal dependecies.
Exploration and check on datas with sql queries (group by, min, max, and count distinct)

If results, the fonctional dependency is incorrect

For table MASTER1, found the fonctional dependies below :
NV -> DEGRE
NV -> CRU
NV -> MILL

NP-> NOM
NP-> PRENOM
NP-> REGION

NV,NP -> QTE

Therefore, one can concieve the following 3 tables in replacement of Master 1 : 

- **Wine**

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    | 
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |


PK : NV
NV is the UID of each wine

- **Producer**

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |


PK : NP
NP is the UID of each producer

- **Produces**

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |

PK : (NV,NP)
NV is a foreign key from table Wine
NP is a foreign Key from table Producer

This table represent the link betwen Producer and their product, ie the production of eah producer... 
In other terms : Producer Produces Wine

Cardinalities : 
- Producer Produces : 1 to many
- Produces Wine : many to 1

Normal Form : 
- 2NF : None of the attributs are dependant of a part of the key
- 3NF : no fonctional dependency betwen non key attributs


#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

In [ ]:
%%sql 
SELECT NP, NV, MIN(QTE) as min1, MAX(Lieu) as max2 
FROM Master1 
group by NP, NV 
having min1 <> max2

In [94]:
%%sql 
SELECT count(*) 
from master2

 * sqlite:///wine.db
Done.


count(*)
185


In [95]:
%%sql 
SELECT NB, NV, LIEU, count(*) 
FROM Master2 
group by NB, NV, LIEU 
having count(*) > 1

 * sqlite:///wine.db
Done.


NB,NV,LIEU,count(*)


Same method as previous, the folloowing dependencies were found : 
NV -> CRU
NV -> DEGRE
NV -> MILL

LIEU-> REGION

NB -> NOM
NB -> PRENOM
NB -> TYPE

DATES,LIEU,NV,NB -> QTE
Functionaly, the df (nv, lieu, nb, date)-> could be questionned except if date endebbed the time (timestamp)

(NOM, PRENOM)-> TYPE (works on the data but would fail if homonymes)

The following tables can be concieved : 

- Wine -> same as for master1


|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |

PK : NV

- Customer


|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |

PK : NB

- Place


|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| LIEU      | Place where the wine was sold                          |
| REGION    | Administrative Region (different to production region) |

PK : LIEU

- Buys


|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |

PK : (NV,LIEU,NB, DATES) 
NV is a foreign key from table Wine
NB is a foreign Key from table Customer
LIEU is a foreign Key from table Place

This table represent the link betwen Customer, Wine and Place, ie the orders of eah customer. 
In other terms : Customer Buys Wine in Place at a certain date
Functionaly, the unicity of primary key (nv, lieu, nb, date) could be questionned

Cardinalities : 
- Customer Buys : 1 to many
- Buys Wine : many to one

Normal Form : 
- 2NF : None of the attributs are dependant of a part of the key
- 3NF : no fonctional dependency betwen non key attributs


Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [7]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

Done.
Done.


[]

In [8]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(NUM)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
dummy:
  0: DEGRE(NUM)


In [9]:
%%sql
SELECT *
FROM dummy;

Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [10]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

In [11]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [35]:
%%sql 
DROP TABLE IF EXISTS WINE;
CREATE TABLE WINE(
    NV INT PRIMARY KEY, 
    CRU VARCHAR NOT NULL, 
    DEGRE INT, M
    ILL NUM NOT NULL);
insert into WINE 
    SELECT DISTINCT NV, CRU, DEGRE, MILL 
    FROM Master1 
    where nv is not null

 * sqlite:///wine.db
Done.
Done.
102 rows affected.


[]

In [43]:
%%
sql select distinct NP NOM, PRENOM, REGION 
from master1 
where np is not null 

 * sqlite:///wine.db
Done.


NOM,PRENOM,REGION


In [47]:
%%sql 
DROP TABLE IF EXISTS PRODUCER;
CREATE TABLE PRODUCER(
    NP INT PRIMARY KEY, 
    NOM VARCHAR NOT NULL, 
    PRENOM VARCHAR, 
    REGION VARCHAR NOT NULL);
insert into PRODUCER 
    select distinct NP, NOM, PRENOM, REGION 
    from master1 
    where np is not null 

 * sqlite:///wine.db
Done.
Done.
124 rows affected.


[]

In [48]:
%sql select * from PRODUCER

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM,REGION
3,Six,Paul,Alsace
6,Marmagne,Bernard,Bourgogne
8,Lioger d'Harduy,Gabriel,Bourgogne
16,Barbin,Bernard,Bourgogne
17,Faiveley,Guy,Bourgogne
18,Tramier,Jean,Bourgogne
19,Dupaquier,Roger,Bourgogne
20,Lamy,Jean,Bourgogne
21,Cornu,Edmond,Bourgogne
26,Violot,Gilbert,Bourgogne


In [55]:
%%sql 
select * 
FROM Master1 
WHERE  (NP is not null and  NV is not null) AND (QTE is null or QTE = 0)

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
34,Montagny,11.2,1978,None,4,Stentz,Fernand,Alsace
68,Riesling,14,1962,None,12,Tortochot,Gabriel,Bourgogne


In [100]:
%%sql 
DROP TABLE IF EXISTS PRODUCES;
CREATE TABLE PRODUCES(NV INT NOT NULL, NP INT NOT NULL, QTE INT, 
                      PRIMARY KEY(NV,NP),
                      FOREIGN KEY (NV) REFERENCES WINE (NV),
                      FOREIGN KEY (NP) REFERENCES WINE (NP));
insert into PRODUCES 
    SELECT DISTINCT NV, NP, QTE 
    FROM MASTER1 
    WHERE  NV IS NOT NULL AND NP IS NOT NULL

 * sqlite:///wine.db
Done.
Done.
140 rows affected.


[]

#### Exercise 1.4

Create the new tables from Master2:

In [97]:
%%sql 
DROP TABLE IF EXISTS CUSTOMER;
CREATE TABLE CUSTOMER(
    NB INT PRIMARY KEY, 
    NOM VARCHAR NOT NULL, 
    PRENOM VARCHAR, 
    TYPE VARCHAR NOT NULL);
insert into CUSTOMER 
    select distinct NB, NOM, PRENOM, TYPE 
    from master2 
    where nb is not null 

 * sqlite:///wine.db
Done.
Done.
100 rows affected.


[]

In [114]:
%%sql DROP TABLE IF EXISTS BUYS;
CREATE TABLE BUYS(
    NV INT NOT NULL, 
    NB INT NOT NULL, 
    LIEU VARCHAR NOT NULL, 
    DATES DATE NOT NULL, 
    QTE INT NOT NULL, 
    PRIMARY KEY(NV,NB,LIEU,DATES),
    FOREIGN KEY (NV) REFERENCES WINE (NV),
    FOREIGN KEY (NB) REFERENCES CUSTOMER (NB));
insert into BUYS 
    select distinct nv, nb, lieu, dates, qte 
    from master2 
    where nv is not null and nb is not null 

 * sqlite:///wine.db
Done.
Done.
73 rows affected.


[]

In [115]:
%%sql 
DROP TABLE IF EXISTS PLACE;
CREATE TABLE PLACE(
    LIEU VARCHAR NOT NULL, 
    REGION VARCHAR NOT NULL,  
    PRIMARY KEY(LIEU),
    FOREIGN KEY (LIEU) REFERENCES BUYS (LIEU));
INSERT INTO PLACE 
    select distinct lieu, region 
    from master2 
    where lieu is not null;

 * sqlite:///wine.db
Done.
Done.
18 rows affected.


[]

# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

In [119]:
print("\nContent of the database")
printSchema(conn)


Content of the database
BUYS:
  0: NV(INT) not null *1
  1: NB(INT) not null *2
  2: LIEU(VARCHAR) not null *3
  3: DATES(DATE) not null *4
  4: QTE(INT) not null
CUSTOMER:
  0: NB(INT) *1
  1: NOM(VARCHAR) not null
  2: PRENOM(VARCHAR)
  3: TYPE(VARCHAR) not null
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
PLACE:
  0: LIEU(VARCHAR) not null *1
  1: REGION(VARCHAR) not null
PRODUCER:
  0: NP(INT) *1
  1: NOM(VARCHAR) not null
  2: PRENOM(VARCHAR)
  3: REGION(VARCHAR) not null
PRODUCES:
  0: NV(INT) not null *1
  1: NP(INT) not null *2
  2: QTE(INT)
WINE:
  0: NV(INT) *1
  1: CRU(VARCHAR) not null
  2: DEGRE(INT)
  3: MILL(NUM) not null


#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [122]:
%%sql 
SELECT TYPE, sum(QTE) as 'Volume des ventes' 
FROM CUSTOMER C JOIN BUYS B ON C.NB = B.NB 
GROUP BY TYPE ORDER BY TYPE;

 * sqlite:///wine.db
Done.


TYPE,Volume des ventes
gros,361
moyen,697
petit,772


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [130]:
%%sql 
SELECT DISTINCT CRU, REGION 
FROM PRODUCER PR JOIN PRODUCES P ON PR.NP= P.NP JOIN WINE W ON W.NV=P.NV
WHERE CRU In ('Pommard','Brouilly')

 * sqlite:///wine.db
Done.


CRU,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [140]:
%%sql 
SELECT DISTINCT REGION 
FROM PRODUCER PR JOIN PRODUCES P ON PR.NP= P.NP  
WHERE NV IN (
    SELECT NV 
    FROM WINE
    WHERE CRU = 'Pommard')
AND REGION IN (SELECT DISTINCT REGION 
FROM PRODUCER PR JOIN PRODUCES P ON PR.NP= P.NP  
WHERE NV IN (
    SELECT NV 
    FROM WINE
    WHERE CRU = 'Brouilly'))

 * sqlite:///wine.db
Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bught by CRU and Millésime

In [141]:
%%sql 
SELECT CRU, MILL, SUM(QTE) as Number_of_wines_bought
FROM WINE W JOIN BUYS B ON B.NV=W.NV
GROUP BY CRU, MILL
ORDER BY CRU, MILL


 * sqlite:///wine.db
Done.


CRU,MILL,Number_of_wines_bought
Arbois,1980,8
Auxey Duresses,1914,80
Beaujolais Primeur,1983,7
Beaujolais Villages,1975,10
Beaujolais Villages,1976,120
Beaujolais Villages,1978,130
Beaujolais Villages,1979,520
Chapelle Chambertin,1973,30
Chateau Corton Grancey,1980,4
Chenas,1984,1


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [143]:
%%sql
SELECT PR.NV, COUNT(DISTINCT NP) AS Number_of_producers
FROM PRODUCES PR JOIN WINE W ON W.NV=PR.NV
GROUP BY PR.NV
HAVING Number_of_producers>3

 * sqlite:///wine.db
Done.


NV,Number_of_producers
45,5
78,5
89,4
98,5


#### Exercise 2.6

Which producers have not produced any wine?

In [144]:
%%sql
SELECT NP, NOM, PRENOM 
FROM PRODUCER
WHERE NP IS NULL OR NP NOT IN (
    SELECT NP FROM PRODUCES)

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [145]:
%%sql SELECT NB, NOM, PRENOM
FROM CUSTOMER
WHERE NB IN (
    SELECT NB 
    FROM BUYS B JOIN WINE W ON W.NV=B.NV
    WHERE MILL = 1980
)

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
8,Aragon,Louis
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [25]:
%%sql 
SELECT NB, NOM, PRENOM
FROM CUSTOMER
EXCEPT
SELECT NB, NOM, PRENOM
FROM CUSTOMER
WHERE  NB  IN (
    SELECT NB 
    FROM BUYS B JOIN WINE W ON W.NV=B.NV
    WHERE MILL = 1980
)


Done.


NB,NOM,PRENOM
1,Aristote,None
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
9,Ajar,Emile
10,Andersen,Yann
11,Breton,Andre
12,Bataille,Georges


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [157]:
%%sql
SELECT NB, NOM, PRENOM
FROM CUSTOMER
WHERE  NB  IN (
    SELECT NB 
    FROM BUYS B JOIN WINE W ON W.NV=B.NV
    WHERE MILL = 1980
)
except
SELECT NB, NOM, PRENOM
FROM CUSTOMER
WHERE  NB  IN (
    SELECT NB 
    FROM BUYS B JOIN WINE W ON W.NV=B.NV
    WHERE MILL <> 1980
)

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


In [155]:
%%sql 

SELECT NB, NOM, PRENOM
FROM CUSTOMER
WHERE  NB  IN (
    SELECT NB 
    FROM BUYS B JOIN WINE W ON W.NV=B.NV
    WHERE MILL = 1980
)
AND NB NOT IN (
SELECT NB
FROM CUSTOMER
WHERE  NB  IN (
    SELECT NB 
    FROM BUYS B JOIN WINE W ON W.NV=B.NV
    WHERE MILL <> 1980
))


 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


#### Exercise 2.10

List all wines from 1980

In [159]:
%%sql
SELECT NV, CRU, MILL, DEGRE 
FROM WINE
WHERE MILL=1980

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5
4,Mercurey,1980,10.9
16,Meursault,1980,12.1
20,Cote de Brouilly,1980,12.1
26,Chateau Corton Grancey,1980,None
28,Volnay,1980,11
43,Fleurie,1980,11.4
74,Arbois,1980,12
78,Etoile,1980,12
79,Seyssel,1980,11


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [161]:
%%sql
SELECT W.NV, CRU, MILL, DEGRE 
FROM WINE W JOIN BUYS B ON B.NV=W.NV
WHERE MILL=1980 AND NB=2

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [162]:
%%sql
SELECT C.NB, NOM, PRENOM
FROM WINE W JOIN BUYS B ON B.NV=W.NV JOIN CUSTOMER C ON C.NB=B.NB
WHERE MILL=1980
GROUP BY C.NB, NOM, PRENOM
HAVING COUNT(distinct W.NV) = (SELECT COUNT(*)
                              FROM WINE 
                               WHERE MILL=1980) 

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
